In [3]:
from PIL import Image
import numpy as np

In [68]:
def crop(ImgArr):
    x_disp = 0; y_disp = 0
    
    i = 0
    while sum(sum(ImgArr[i])) == 0:
        ImgArr = np.delete(ImgArr, i, 0)
    i = -1
    while sum(sum(ImgArr[i])) == 0:
        ImgArr = np.delete(ImgArr, i, 0)
        y_disp += 1
    i = 0
    while sum(sum(ImgArr[:,i])) == 0:
        ImgArr = np.delete(ImgArr, i, 1)
    i = -1
    while sum(sum(ImgArr[:,i])) == 0:
        ImgArr = np.delete(ImgArr, i, 1)
        x_disp += 1
    return ImgArr, x_disp, y_disp

In [69]:
ALPHA = "ALPHA"
alpha_bit = 0xFE

def RGBA_to_bitmap(col):
    #8 BIT: BBBGGGRR
    if len(col) == 4:
        if col[3] == 0: return ALPHA
    col = np.array(col)/255.
    bitcol = int(round(col[0]*3) + round(col[1]*7)*4 + round(col[2]*7)*4*8)
    if bitcol == alpha_bit: bitcol += 1
    return bitcol

def bitmap_to_RGBA(bitcol):
    #8 BIT: BBBGGGRR
    if bitcol == ALPHA: return [0,0,0,0]
    R = bitcol%4
    G = bitcol//4 % 8
    B = bitcol//(4*8)
    return [int(x) for x in [R/3.*255, G/7.*255, B/7.*255, 255]]
    
def GetBitmap(ImgArr):
    shape = ImgArr.shape
    Bitmap = [[RGBA_to_bitmap(COL) for COL in row] for row in ImgArr]
    return {"X": shape[1], "Y": shape[0]}, Bitmap

def Bitmap_to_string(Bitmap):
    return ", ".join([str(item) for sublist in Bitmap for item in sublist])

def Bitmap_to_image(Bitmap):
    return Image.fromarray(np.array([[bitmap_to_RGBA(COL) for COL in row] for row in Bitmap], dtype="uint8"))

In [70]:
def Filename_to_bitmap(FILENAME, RESOLUTION_x, Orientation = "xy"):
    '''
    FILENAME must be of an image with an alpha channel
    RESOLUTION_x is an estimate of the x-resolution of the final bitmap
    
    returns:
        STRING, string of final bitmap. Just copy and paste it
        shape: shape of final bitmap (after cropping extra alpha rows and columns)
    '''
    
    Img = Image.open(FILENAME)
    RX = np.asarray(Img).shape[0]
    RY = np.asarray(Img).shape[1]
    if Orientation == "yx": t = RX; RX = RY; RY = t;
    scale = RESOLUTION_x/RX
    Img = Img.resize((round(RX*scale), round(RY*scale)), Image.NEAREST)
    ImgArr = np.asarray(Img)
    ImgArrCrop, x_disp, y_disp = crop(ImgArr)
    shape, Bitmap = GetBitmap(ImgArrCrop)
    Bitmap = [[col if col != 0 else 0xFF for col in row] for row in Bitmap]
    STRING = Bitmap_to_string(Bitmap)
    preview = Bitmap_to_image(Bitmap)
    
    return STRING, shape, preview, x_disp, y_disp

In [71]:
FILENAME = r"C:\Users\nailu\Downloads\eheh.png"
#FILENAME = r"C:\Users\nailu\Downloads\grass.png"
#FILENAME = r"C:\Users\nailu\Downloads\flag (3).jpg"
#FILENAME = r"C:\Users\nailu\Downloads\hamburger_menu.png"
STRING, shape, preview, x_disp, y_disp = Filename_to_bitmap(FILENAME, 30, Orientation = "yx")

In [72]:
shape

{'X': 17, 'Y': 27}

In [73]:
preview

In [74]:
STRING

'ALPHA, ALPHA, ALPHA, 210, 210, ALPHA, ALPHA, ALPHA, ALPHA, ALPHA, 133, 133, ALPHA, ALPHA, ALPHA, ALPHA, ALPHA, ALPHA, ALPHA, 215, 210, 210, 211, ALPHA, ALPHA, ALPHA, 165, 133, 133, 165, ALPHA, ALPHA, ALPHA, ALPHA, ALPHA, ALPHA, 211, 211, 210, 211, 210, ALPHA, 165, 97, 165, 133, 133, 165, ALPHA, ALPHA, ALPHA, ALPHA, 210, 215, 211, 211, 210, 97, 210, 97, 165, 197, 165, 133, 165, ALPHA, ALPHA, ALPHA, ALPHA, 210, 210, 243, 243, 243, 243, 97, 97, 197, 197, 193, 165, 133, 165, ALPHA, ALPHA, ALPHA, ALPHA, 243, 170, 243, 243, 243, 243, 243, 234, 234, 234, 165, 133, 133, 165, ALPHA, ALPHA, 211, 174, 243, 243, 243, 243, 243, 243, 243, 206, 202, 206, 165, 133, 165, ALPHA, ALPHA, 243, 243, 243, 243, 211, 211, 251, 251, 251, 251, 251, 251, 165, 165, 64, 137, 211, 243, 243, 243, 243, 211, 255, 255, 255, 255, 255, 255, 255, 255, 133, 64, 137, 243, 243, 243, 243, 211, 255, 255, 32, 36, 36, 255, 255, 255, 64, 64, 137, 64, 243, 243, 243, 243, 211, 251, 36, 36, 255, 255, 36, 255, 36, 36, 255, 64, 64, 24

In [75]:
from PIL import GifImagePlugin

In [76]:
def replace_alpha(col, frame):
    frame = frame.copy();
    for i in range(frame.shape[0]):
        for j in range(frame.shape[1]):
            elif (sum([1 for X,Y in zip(frame[i][j], col) if X != Y]) == 0):
                for k in range(4): frame[i][j][k] = 0
    return frame

SyntaxError: invalid syntax (<ipython-input-76-d83ca0e7f3be>, line 5)

In [84]:
FILENAME = r"./cutie2.gif"
SPRITENAME = "RUN"
RESOLUTION_X = 60 

GIF = Image.open(FILENAME)
n_frames = GIF.n_frames
FRAMES = []
for i in range(n_frames):
    GIF.seek(i)
    img = GIF.copy()
    FRAMES.append(img)
    
scale = float(RESOLUTION_X)/np.asarray(FRAMES[0]).shape[1]
FRAMES_resized = [Img.resize((int(np.asarray(Img).shape[1] * scale), int(np.asarray(Img).shape[0] * scale)), Image.NEAREST) for Img in FRAMES]
FRAMES_RGBA = [np.asarray(Img.convert("RGBA")) for Img in FRAMES_resized]
FRAMES_RGBA = [replace_alpha([192, 192, 192, 255], frame) for frame in FRAMES_RGBA]
FRAMES_cropped = [crop(Img)[0] for Img in FRAMES_RGBA]
FRAMES_disp = [(crop(Img)[1], crop(Img)[2]) for Img in FRAMES_RGBA]
FRAMES_disp = [(disp[0] - FRAMES_disp[0][0], disp[1] - FRAMES_disp[0][1]) for disp in FRAMES_disp]
Bitmap = [GetBitmap(Img) for Img in FRAMES_cropped]
STRING = [(shape, Bitmap_to_string(bitmap)) for shape, bitmap in Bitmap]

print(Bitmap[0][0])

DISP_STR = "{" + ", ".join(["{0}, {1}".format(disp[0], disp[1]) for disp in FRAMES_disp]) + "};"
CODE = '''const char {0}_disp[{1}] PROGMEM = {2}\n'''.format(SPRITENAME, len(STRING)*2, DISP_STR)
CODE += '''const unsigned char {0}_XY[{1}] PROGMEM = '''.format(SPRITENAME, len(STRING)*2) + "{" + ", ".join(["{0}, {1}".format(shape['X'],shape['Y']) for shape, _ in Bitmap]) + "};" + '''
''' + "\n".join(["const unsigned char {2}_{0}[{3}] PROGMEM = {1} ".format(i, "{" + string[1] + "};", SPRITENAME, Bitmap[i][0]['X']*Bitmap[i][0]['Y']) for i, string in enumerate(STRING)]) + '''

const long {0}_ARR[{1}] PROGMEM = '''.format(SPRITENAME, len(STRING)) + "{" + ", ".join(["{0}_{1}".format(SPRITENAME, i) for i in range(len(STRING))]) + "};"

with open("{}_code.txt".format(SPRITENAME), "w") as f: f.write(CODE)

{'X': 20, 'Y': 25}


In [79]:
import os

FOLDER = r".\kaho_kick"
SPRITENAME = "kaho_kick"
RESOLUTION_X = 30

Img_Name_List = os.listdir(FOLDER)
FRAMES = [Image.open(FOLDER + "\\" + img) for img in Img_Name_List]
    
scale = float(RESOLUTION_X)/np.asarray(FRAMES[0]).shape[1]
FRAMES_resized = [Img.resize((int(np.asarray(Img).shape[1] * scale), int(np.asarray(Img).shape[0] * scale)), Image.NEAREST) for Img in FRAMES]
FRAMES_RGBA = [np.asarray(Img.convert("RGBA")) for Img in FRAMES_resized]
FRAMES_cropped = [crop(Img)[0] for Img in FRAMES_RGBA]
FRAMES_disp = [(crop(Img)[1], crop(Img)[2]) for Img in FRAMES_RGBA]
FRAMES_disp = [(disp[0] - FRAMES_disp[0][0], disp[1] - FRAMES_disp[0][1]) for disp in FRAMES_disp]
Bitmap = [GetBitmap(Img) for Img in FRAMES_cropped]
STRING = [(shape, Bitmap_to_string(bitmap)) for shape, bitmap in Bitmap]

print(Bitmap[0][0])

DISP_STR = "{" + ", ".join(["{0}, {1}".format(disp[0], disp[1]) for disp in FRAMES_disp]) + "};"
CODE = '''const char {0}_disp[{1}] PROGMEM = {2}\n'''.format(SPRITENAME, len(STRING)*2, DISP_STR)
CODE += '''const unsigned char {0}_XY[{1}] PROGMEM = '''.format(SPRITENAME, len(STRING)*2) + "{" + ", ".join(["{0}, {1}".format(shape['X'],shape['Y']) for shape, _ in Bitmap]) + "};" + '''
''' + "\n".join(["const unsigned char {2}_{0}[{3}] PROGMEM = {1} ".format(i, "{" + string[1] + "};", SPRITENAME, Bitmap[i][0]['X']*Bitmap[i][0]['Y']) for i, string in enumerate(STRING)]) + '''

const long {0}_ARR[{1}] PROGMEM = '''.format(SPRITENAME, len(STRING)) + "{" + ", ".join(["{0}_{1}".format(SPRITENAME, i) for i in range(len(STRING))]) + "};"

with open("{}_code.txt".format(SPRITENAME), "w") as f: f.write(CODE)

{'X': 12, 'Y': 22}


In [80]:
from functools import reduce 
reduce(lambda a,b: a*b, [2,3,4])

24